<h3>Load model data and predict</h3>

In [1]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [2]:
#load YAML
with open('data.yaml', mode='r') as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)

labels = data_yaml['names']

#load YOLO model
yolo_model = cv2.dnn.readNetFromONNX('./Model/weights/best.onnx')
yolo_model.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo_model.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [3]:
#load image
img = cv2.imread('./street_image.jpg')
image = img.copy()
row, col, d = image.shape

#modify input 
max_rc = max(row, col)
input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
input_image[0:row, 0:col] = image

#predict from input using yolo model
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB = True, crop = False)
yolo_model.setInput(blob)
predictions = yolo_model.forward()

cv2.imshow('image', input_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
predictions.shape

(1, 25200, 25)

<img src="yolo_pred_format.png" width=700 height=400>

In [5]:
# #Non max suppression
# #filter those with confidence more than 0.4 and probability more than 0.25
# conf_threshold = 0.4
# prob_threshold = 0.25
# detections = predictions[0]     

# #width and height of image
# image_w, image_h = input_image.shape[:2]
# x_factor = image_w/INPUT_WH_YOLO
# y_factor = image_h/INPUT_WH_YOLO

# #filtering according to thresholds
# detections = detections[detections[:, 4] > conf_threshold]
# pred_cols = detections[:, 6:]
# pred_col = np.max(pred_cols, axis=1, keepdims=True)
# max_index = np.argmax(pred_cols, axis=1, keepdims=True)
# detections = detections[:,0:6] + pred_col
# detections = detections[detections[:, 5] > prob_threshold]

# column_indices = [0, 1, 2, 3]  
# cx, cy, w, h = np.hsplit(detections[:, column_indices], 4)

# # bounding box values
# # left, top, width and height
# left = ((cx - 0.5*w)*x_factor).astype(int)
# top = ((cy - 0.5*h)*y_factor).astype(int)
# width = (w*x_factor).astype(int)
# height = (h*y_factor).astype(int)

# boxes = np.hstack((left, top, width, height))
# confidences = np.array(np.hsplit(detections[:, 4], 1)).T
# classes = max_index

# #clean data
# classes = classes.flatten().tolist()
# boxes = boxes.tolist()
# confidences = confidences.flatten().tolist()

In [6]:
# #Draw boundart boxes
# for idx in index:
#     x, y, w, h = boxes[idx]
#     bb_conf = int(confidences[idx]*100)
#     class_id = classes[idx]
#     class_name = labels[class_id]
    
#     text = f'{class_name}: {bb_conf}%'
    
#     cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
#     cv2.rectangle(image, (x, y-30), (x+w, y), (255, 255, 255), -1)
    
#     cv2.putText(image, text, (x, y-10), cv2.FONT_HERSHEY_PLAIN, 0.76, (0,0,0), 1)
    

In [7]:
#Non max suppression
# index = cv2.dnn.NMSBoxes(boxes, confidences, 0.25, 0.4)

In [8]:
# Non Maximum Supression
# step-1: filter detection based on confidence (0.4) and probability score (0.25)
detections = predictions[0]
boxes = []
confidences = []
classes = []

# widht and height of the image (input_image)
image_w, image_h = input_image.shape[:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] # confidence of detection an object
    if confidence > 0.4:
        class_score = row[5:].max() # maximum probability from 20 objects
        class_id = row[5:].argmax() # get the index position at which max probabilty occur
        
        if class_score > 0.25:
            cx, cy, w, h = row[0:4]
            # construct bounding from four values
            # left, top, width and height
            left = int((cx - 0.5*w)*x_factor)
            top = int((cy - 0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)
            
            box = np.array([left,top,width,height])
            
            # append values into the list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)
            
# clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

# NMS
index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45).flatten()


In [9]:
# Draw the Bounding
for ind in index:
    # extract bounding box
    x,y,w,h = boxes_np[ind]
    bb_conf = int(confidences_np[ind]*100)
    classes_id = classes[ind]
    class_name = labels[classes_id]
    
    text = f'{class_name}: {bb_conf}%'
    
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.rectangle(image,(x,y-30),(x+w,y),(255,255,255),-1)
    
    cv2.putText(image,text,(x,y-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)
    


In [10]:
cv2.imshow('original', img)
cv2.imshow('yolo_prediction', image)
cv2.waitKey(0)
cv2.destroyAllWindows()